In [2]:
#model 
from Triplet_Network import TripletResNet_features
import torch
import torch.nn as nn
import pandas as pd
from create_data_loader import TripletLossDataset_features, column_list_to_tensor
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')
dataset_train = torch.load('DATA/Dataset_toload/train_dataset_random.pt')
df = pd.read_csv('DATA/Dataset/wikiart_full_combined.csv')
feature = 'image_features'
df[feature] = column_list_to_tensor(df,feature)
df = df[:5000]
# model = TripletResNet_features(dataset_train.dimension).to(device)
# model_path = 'trained_models/TripletResNet_random_5000.pth'
# model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
# model.eval()
# with torch.no_grad():
#     df['trained_features'] = df[feature].apply(lambda x: model.forward_once(x.to(device)))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: traopia. Use `wandb login --relogin` to force relogin


In [14]:
import itertools
import json 
import faiss
import numpy as np

with open('DATA/influenced_by_dict.json', 'r') as fp:
    dict_influence = json.load(fp)

if 'val' in df['mode'].unique():
    val_indices = df[df['mode'] == 'val'].index

def vector_similarity_search( query, index_list,num_examples, feature,df):
    '''Search for similar vectors in the dataset using faiss library'''
    k = num_examples+1
    query_vector = torch.stack(df[feature].tolist())[query].reshape(1, -1)
    xb = torch.stack(df[feature].tolist())[index_list]
    d = xb.shape[1]
    index = faiss.IndexFlatL2(d) 
    index.add(xb)
    D, I = index.search(query_vector, k)
    I = list(I[0][1:]) 
    I = [i.item() for i in I]
    I = [index_list[i] for i in I]
    return I

def check_chain_influence(query,df,dict_influence, closest_indices):
    influencers_query = df.iloc[query].influenced_by
    influencers_query = influencers_query.split(',')
    influencers_query = [x.strip() for x in influencers_query]
    influencers_query = [x for x in influencers_query if x in dict_influence.keys()]

    dict_influence_query = {k: dict_influence[k] for k in influencers_query if k in dict_influence}
    second_order_influencers = list(itertools.chain.from_iterable(list(dict_influence_query.values())))
    third_order_influencers = list(itertools.chain.from_iterable([dict_influence[x] for x in second_order_influencers if x in dict_influence]))
    all_influencers = influencers_query + second_order_influencers + third_order_influencers
    all_influencers = list(set(all_influencers))
    
    closest_indices_artists = [df.iloc[x].artist_name for x in closest_indices]
    result = [x for x in all_influencers if x in closest_indices_artists ]

    #count the number of closest indices whose artist name is in result
    count = sum(df.iloc[x].artist_name in result for x in closest_indices)
    return count, result

In [10]:

    
average = []    
for query in val_indices:
    indices = df[df.artist_name != df.iloc[query].artist_name].index
    general_positive_examples = vector_similarity_search(query, indices, 10, 'image_features',df)
    count, result = check_chain_influence(query,df,dict_influence, general_positive_examples)
    average.append(count)
    print(count, result)
    print('-------------------')
np.mean(average)



    

0 []
-------------------
0 []
-------------------
9 ['eugene-boudin', 'camille-pissarro', 'charles-francois-daubigny', 'pierre-auguste-renoir', 'camille-corot']
-------------------
3 ['pierre-auguste-renoir', 'diego-velazquez', 'domenico-ghirlandaio']
-------------------
4 ['william-turner', 'pietro-perugino', 'camille-pissarro', 'peter-paul-rubens']
-------------------
4 ['rembrandt', 'paolo-veronese', 'peter-paul-rubens']
-------------------
0 []
-------------------
0 []
-------------------
5 ['diego-velazquez', 'camille-corot', 'johannes-vermeer', 'anthony-van-dyck']
-------------------
5 ['diego-velazquez', 'peter-paul-rubens', 'frans-hals', 'anthony-van-dyck']
-------------------
4 ['edouard-manet', 'diego-velazquez', 'frans-hals']
-------------------
4 ['diego-velazquez', 'rembrandt', 'anthony-van-dyck']
-------------------
0 []
-------------------
0 []
-------------------
0 []
-------------------
0 []
-------------------
2 ['peter-paul-rubens', 'anthony-van-dyck']
--------------

2.543340380549683

In [6]:
model = TripletResNet_features(dataset_train.dimension)
model_path = 'trained_models/TripletResNet_random_5000.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()
with torch.no_grad():
    df['trained_features'] = df[feature].apply(lambda x: model.forward_once(x))

In [15]:

    
average = []    
for query in val_indices:
    indices = df[df.artist_name != df.iloc[query].artist_name].index
    general_positive_examples = vector_similarity_search(query, indices, 10, 'trained_features',df)
    count, result = check_chain_influence(query,df,dict_influence, general_positive_examples)
    average.append(count)
    print(count, result)
    print('-------------------')
np.mean(average)



    

/Users/traopia/miniconda3/envs/artsagenet_new/lib/python3.11/site-packages/faiss/contrib/torch_utils.py:51: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  x.storage().data_ptr() + x.storage_offset() * 4)


0 []
-------------------
1 ['hans-holbein-the-younger']
-------------------
8 ['camille-pissarro', 'pierre-auguste-renoir', 'claude-monet', 'eugene-boudin']
-------------------
3 ['pierre-auguste-renoir']
-------------------
5 ['gustave-courbet', 'camille-pissarro', 'diego-velazquez', 'william-turner']
-------------------
4 ['eugene-boudin', 'camille-corot', 'camille-pissarro', 'william-turner']
-------------------
0 []
-------------------
3 ['rembrandt', 'charles-francois-daubigny']
-------------------
8 ['titian', 'rembrandt', 'peter-paul-rubens', 'hans-holbein-the-younger', 'caravaggio']
-------------------
5 ['paolo-veronese', 'peter-paul-rubens', 'hans-holbein-the-younger', 'andrea-mantegna', 'caravaggio']
-------------------
1 ['gustave-courbet']
-------------------
9 ['gustave-courbet', 'titian', 'frans-hals', 'rembrandt', 'paolo-veronese', 'peter-paul-rubens', 'diego-velazquez']
-------------------
0 []
-------------------
0 []
-------------------
0 []
-------------------
0 []


3.4989429175475686